# Part 4b - Model testing locally
 

In [1]:
!mkdir model

In [2]:
!aws s3 cp s3://sagemaker-us-east-1-905847418383/huggingface-pytorch-training-2021-12-01-09-15-45-087/output/model.tar.gz model/

download: s3://sagemaker-us-east-1-905847418383/huggingface-pytorch-training-2021-12-01-09-15-45-087/output/model.tar.gz to model/model.tar.gz


In [3]:
!tar -xf model/model.tar.gz -C model/

In [4]:
!rm -r model/checkpoint*

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [31]:
tokenizer = AutoTokenizer.from_pretrained('./model/')

In [33]:
model = AutoModelForSeq2SeqLM.from_pretrained('./model/').to('cpu').eval()

In [34]:
texts[0]

"  Consider the blow-up $X$ of $\\mathbb{P}^3$ at 6 points in very general position and the 15 lines through the 6 points. We construct an infinite-order pseudo-automorphism $\\phi_X$ on $X$, induced by the complete linear system of a divisor of degree 13. The effective cone of $X$ has infinitely many extremal rays and hence, $X$ is not a Mori Dream Space. The threefold $X$ has a unique anticanonical section which is a Jacobian K3 Kummer surface $S$ of Picard number 17. The restriction of $\\phi_X$ on $S$ realizes one of Keum's 192 infinite-order automorphisms of Jacobian K3 Kummer surfaces. In general, we show the blow-up of $\\mathbb{P}^n$ ($n\\geq 3$) at $(n+3)$ very general points and certain 9 lines through them is not Mori Dream, with infinitely many extremal effective divisors. As an application, for $n\\geq 7$, the blow-up of $\\overline{M}_{0,n}$ at a very general point has infinitely many extremal effective divisors. "

In [35]:
input_ids = tokenizer(texts[0], truncation=True, padding='longest', return_tensors="pt").input_ids.to('cpu')

In [36]:
output = model.generate(input_ids)
predictions = tokenizer.batch_decode(output, skip_special_tokens=True)

In [37]:
predictions

['The blow-up of $\\mathbb{P}^3$ at 6 points and certain 9 lines through   them is not Mori Dream, with infinitely many extremal effective divisors and applications to the blow-ups of $\\overline{M}_{0,n}$ at a very general point']

In [ ]:
import pandas as pd
df_test = pd.read_csv('data/test.csv')
ref_summaries = list(df_test['summary'])
texts = list(df_test['text'])

In [20]:
from datasets import load_metric
metric = load_metric("rouge")

In [21]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [22]:
calc_rouge_scores(candidate_summaries, ref_summaries)

{'rouge1': 44.5, 'rouge2': 24.5, 'rougeL': 39.4, 'rougeLsum': 39.4}

In [31]:
candidate_summaries_topk = []

for i, text in enumerate(texts):
    if i % 100 == 0:
        print(i)
    data = {"inputs":text, "parameters_list":[{"min_length": 5, "max_length": 20, "num_beams": 50, "top_p": 0.9, "do_sample": True}]}
    candidate = predictor.predict(data)
    candidate_summaries_topk.append(candidate[0][0])

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [13]:
file = open("model-summaries.txt", "w")
for s in candidate_summaries:
    file.write(s + "\n")
file.close()